# Phyllosphere of Hawaiian Plants - Fungal ITS Analysis

Note that the sequence files and the qiime2 files containing all the sequences (demux-paired-end.qza) and the trimmed ITS sequences (trimmed.qza) are not provided inn the repository because of their size. The steps below allow you to recreate the files.

## Import Sequences into Qiime2

The sequences for ITS amplicon sequencing of the 47 samples should be downloaded from GenBank's SRA database under Bioproject PRJNA880325. The titles of Biosamples containing the ITS reads start with "ITS2".

Once downloaded the files should be moved to a directory called "Data" within the current directory.

In [ ]:
!mkdir 01-ProcessReads
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path 01-ProcessReads/demux-paired-end.qza

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/demux-paired-end.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/demux.qzv

## Trim ITS using itsxpress:

In [ ]:
!qiime itsxpress trim-pair-output-unmerged\
  --i-per-sample-sequences 01-ProcessReads/demux-paired-end.qza \
  --p-region ITS1 \
  --p-taxa F \
  --p-cluster-id 1.0 \
  --p-threads 30 \
  --o-trimmed 01-ProcessReads/trimmed.qza

In [ ]:
!qiime demux summarize \
 --i-data 01-ProcessReads/trimmed.qza \
 --p-n 100000 \
 --verbose \
 --o-visualization 01-ProcessReads/trimmed.qzv

In [ ]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs 01-ProcessReads/trimmed.qza \
  --p-trunc-len-r 200 \
  --p-trunc-len-f 0 \
  --p-n-threads 30 \
  --o-representative-sequences 01-ProcessReads/rep-seqs-dada.qza \
  --o-table 01-ProcessReads/table-dada.qza \
  --o-denoising-stats 01-ProcessReads/stats-dada.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/stats-dada.qza \
  --o-visualization 01-ProcessReads/stats-dada.qzv

!qiime feature-table summarize \
  --i-table 01-ProcessReads/table-dada.qza \
  --o-visualization 01-ProcessReads/table-dada.qzv \
  --m-sample-metadata-file map.txt

## Make Unite classifier

Unite version 8.3
https://unite.ut.ee/repository.php#uchime

Downloaded from: https://doi.org/10.15156/BIO/1264763


https://files.plutof.ut.ee/public/orig/B3/05/B3054DF783AC61A0C3BD0FDEB0516EC394934809AAE43CA0F3081C0A184FAA39.tgz

In [ ]:
!wget https://files.plutof.ut.ee/public/orig/B3/05/B3054DF783AC61A0C3BD0FDEB0516EC394934809AAE43CA0F3081C0A184FAA39.tgz

In [ ]:
!gunzip B3054DF783AC61A0C3BD0FDEB0516EC394934809AAE43CA0F3081C0A184FAA39.tgz

In [ ]:
!tar -xvf B3054DF783AC61A0C3BD0FDEB0516EC394934809AAE43CA0F3081C0A184FAA39.tar

In [ ]:
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path sh_qiime_release_s_10.05.2021/sh_refs_qiime_ver8_dynamic_s_10.05.2021.fasta \
  --output-path unite.qza

In [ ]:
!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path sh_qiime_release_s_10.05.2021/sh_taxonomy_qiime_ver8_dynamic_s_10.05.2021.txt \
  --output-path unite-taxonomydev.qza

# Assign Taxonomy

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads unite.qza \
  --i-reference-taxonomy unite-taxonomydev.qza \
  --o-classifier classifierdev.qza

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier classifierdev.qza \
  --i-reads 01-ProcessReads/rep-seqs-dada.qza \
  --p-n-jobs 30 \
  --o-classification 01-ProcessReads/taxonomydev.qza

In [ ]:
!qiime metadata tabulate \
  --m-input-file 01-ProcessReads/taxonomydev.qza \
  --o-visualization 01-ProcessReads/taxonomydev.qzv

## Analyses

In [ ]:
# Taxa plots
!qiime taxa barplot \
  --i-table 01-ProcessReads/table-dada.qza \
  --i-taxonomy 01-ProcessReads/taxonomydev.qza \
  --m-metadata-file map.txt \
  --o-visualization 02-Analyses/taxa-bar-plots-sklearndev.qzv


In [ ]:
!qiime diversity core-metrics \
  --i-table 01-ProcessReads/table-dada.qza \
  --p-sampling-depth 19000 \
  --m-metadata-file map.txt \
  --output-dir 02-Analyses/core-metrics-results-vsearch